In [237]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
import joblib
import csv

song_data = pd.read_csv('song-data.csv')

listening_ohe = OneHotEncoder()
listening_arrays = listening_ohe.fit_transform(song_data[['Listening To Song','Artist', 'Genre']]).toarray()
listening_labels = []

for labels in listening_ohe.categories_:
    for label in labels:
        listening_labels.append(label)


recommended_ohe = OneHotEncoder(handle_unknown='ignore')
recommended_arrays = recommended_ohe.fit_transform(song_data[['Recommended Song','Artist.1', 'Genre.1']]).toarray()
recommended_labels = []

for labels in recommended_ohe.categories_:
    for label in labels:
        recommended_labels.append(label)
        
listening_dataframe = pd.DataFrame(listening_arrays, columns = listening_labels)
recommended_dataframe = pd.DataFrame(recommended_arrays, columns = recommended_labels)

X = listening_dataframe
y = recommended_dataframe

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# joblib.dump(model, 'song-recommender.joblib')

predictions = model.predict(X_test)

inverse_listening = listening_ohe.inverse_transform(X_test)
inverse_recommended = recommended_ohe.inverse_transform(predictions)

f1 = pd.DataFrame(inverse_listening, columns=['Listening','Artist','Genre'])
f2 = pd.DataFrame(inverse_recommended, columns=['Song','Artist','Genre'])

merged_results = f1.merge(f2, left_on='Listening', right_on='Song')
merged_results.to_csv('recommended-songs.csv')


# tree.export_graphviz(model, out_file='next-song-decision.dot',
#                     feature_names=recommended_labels,
#                     class_names=sorted(recommended_labels),
#                     label=all,
#                     rounded=True,
#                     filled=True)